In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('OnlineRetail.csv',encoding='latin-1')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [2]:
#Ques1
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
start_date = '2011-09-01'
end_date = '2011-11-30'
filtered_df = df[(df['InvoiceDate'] >= start_date) & (df['InvoiceDate'] <= end_date)]
filtered_df.to_csv('12012036_12012114.csv', index=False)

In [3]:
filtered_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
49034,540498,21485,RETROSPOT HEART HOT WATER BOTTLE,1,2011-09-01 10:06:00,4.95,17243.0,United Kingdom
49035,540498,22112,CHOCOLATE HOT WATER BOTTLE,1,2011-09-01 10:06:00,4.95,17243.0,United Kingdom
49036,540498,17091A,LAVENDER INCENSE IN TIN,6,2011-09-01 10:06:00,1.25,17243.0,United Kingdom
49037,540498,84813,SET OF 4 DIAMOND NAPKIN RINGS,6,2011-09-01 10:06:00,12.75,17243.0,United Kingdom
49038,540498,22795,SWEETHEART RECIPE BOOK STAND,3,2011-09-01 10:06:00,6.75,17243.0,United Kingdom


In [4]:
#Ques2
invoice_df = pd.DataFrame()
grouped = df.groupby('InvoiceNo')['StockCode'].apply(list).reset_index()
invoice_df['InvoiceNo'] = grouped['InvoiceNo']
invoice_df['StockCodes'] = grouped['StockCode']
invoice_df.to_csv('invoice_data_12012036_12012114.csv', index=False)

In [5]:
invoice_df.head()

,InvoiceNo,StockCodes
0,536365,"[85123A, 71053, 84406B, 84029G, 84029E, 22752,..."
1,536366,"[22633, 22632]"
2,536367,"[84879, 22745, 22748, 22749, 22310, 84969, 226..."
3,536368,"[22960, 22913, 22912, 22914]"
4,536369,[21756]


In [6]:
#Ques3
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import ast
new_df=pd.read_csv('invoice_data_12012036_12012114.csv')
transactions = new_df['StockCodes'].apply(lambda x: x.split(','))

transactions[0:5]

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = apriori(df_encoded, min_support=0.01, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

filtered_rules = rules[(rules['support'] >= 0.01) & (rules['lift'] > 1.0)]

print(filtered_rules)

              antecedents           consequents  antecedent support  \
0             ( '85099B')            ( '20711')            0.073591   
1              ( '20711')           ( '85099B')            0.017606   
2              ( '20712')            ( '21928')            0.031120   
3              ( '21928')            ( '20712')            0.028571   
4              ( '20712')            ( '21929')            0.031120   
..                    ...                   ...                 ...   
879  ( '22728',  '22727')            ( '22726')            0.016757   
880  ( '22726',  '22727')            ( '22728')            0.022239   
881            ( '22728')  ( '22726',  '22727')            0.029498   
882            ( '22726')  ( '22728',  '22727')            0.035946   
883            ( '22727')  ( '22728',  '22726')            0.038494   

     consequent support   support  confidence       lift  leverage  \
0              0.017606  0.011274    0.153200   8.701515  0.009978   
1      

In [7]:
#Ques4
top_10_rules = filtered_rules.sort_values(by='confidence', ascending=False).head(10)

print(top_10_rules)

              antecedents  consequents  antecedent support  \
507            ( '23172')   ( '23171')            0.011506   
872  ( '22699',  '22698')   ( '22697')            0.020734   
656  ( '20723',  '22356')   ( '20724')            0.014015   
861  ( '22423',  '22698')   ( '22697')            0.012201   
435            ( '22579')   ( '22578')            0.012471   
638  ( '20719',  '22356')   ( '20724')            0.014286   
873  ( '22697',  '22698')   ( '22699')            0.021776   
796  ( '22386',  '21928')  ( '85099B')            0.013475   
588           ( '84596F')  ( '84596B')            0.016062   
650  ( '20723',  '22355')   ( '20724')            0.015135   

     consequent support   support  confidence       lift  leverage  \
507            0.014054  0.010000    0.869128  61.841766  0.009838   
872            0.037259  0.017645    0.851024  22.840961  0.016872   
656            0.038224  0.011815    0.842975  22.053594  0.011279   
861            0.037259  0.010270    

In [8]:
#Ques5
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
start_date = '2010-12-01'
end_date = '2011-03-03'
country = 'France'

filtered_df = df[(df['InvoiceDate'] >= start_date) & (df['InvoiceDate'] <= end_date) & (df['Country'] == country)]
basket = (filtered_df.groupby(['InvoiceNo', 'StockCode'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket_sets = basket.applymap(lambda x: 1 if x > 0 else 0)

frequent_item_sets = apriori(basket_sets, min_support=0.04, use_colnames=True)

association_rules_df = association_rules(frequent_item_sets, metric='confidence', min_threshold=0.0)

top_10_rules = association_rules_df.sort_values(by='confidence', ascending=False).head(10)

print(top_10_rules)

               antecedents    consequents  antecedent support  \
889  (22303, 21559, 22554)         (POST)            0.051948   
728         (22728, 22727)         (POST)            0.051948   
716         (22726, 22727)         (POST)            0.051948   
718                (22726)  (POST, 22727)            0.051948   
719                (22727)  (POST, 22726)            0.051948   
721          (POST, 22726)        (22728)            0.051948   
722         (22728, 22726)         (POST)            0.051948   
725                (22726)  (POST, 22728)            0.051948   
727          (POST, 22727)        (22728)            0.051948   
731                (22727)  (POST, 22728)            0.051948   

     consequent support   support  confidence   lift  leverage  conviction  \
889            0.571429  0.051948         1.0   1.75  0.022263         inf   
728            0.571429  0.051948         1.0   1.75  0.022263         inf   
716            0.571429  0.051948         1.0   1.

/Users/shambhavisingh/anaconda3/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [9]:
#Ques6
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
ratings_df = pd.read_csv('ratings.csv',encoding='latin-1')  
movies_df = pd.read_csv('movies.csv',encoding='latin-1')  

action_genre_movies = movies_df[movies_df['genres'].str.contains("Action")]
pivot_table = pd.pivot_table(ratings_df, index='userId', columns='movieId', values='rating')

movie_similarities = {}

for movie in action_genre_movies['movieId']:
    movie_ratings = pivot_table[movie]
    for other_movie in action_genre_movies['movieId']:
        if movie != other_movie:
            other_movie_ratings = pivot_table[other_movie]
            common_users = movie_ratings.dropna().index.intersection(other_movie_ratings.dropna().index)
            
            if len(common_users) >= 2:
                pearson_coefficient, _ = pearsonr(movie_ratings[common_users], other_movie_ratings[common_users])
                movie_similarities[(movie, other_movie)] = pearson_coefficient

heat_movie_id = movies_df[movies_df['title'] == 'Heat']['movieId'].values[0]
heat_similar_movies = sorted(movie_similarities.keys(), key=lambda x: -movie_similarities[x])
heat_top_similar = [movies_df[movies_df['movieId'] == movie_id[1]]['title'].values[0] for movie_id in heat_similar_movies[:5]]

eraser_movie_id = movies_df[movies_df['title'] == 'Eraser']['movieId'].values[0]
eraser_similar_movies = sorted(movie_similarities.keys(), key=lambda x: -movie_similarities[x])
eraser_top_similar = [movies_df[movies_df['movieId'] == movie_id[1]]['title'].values[0] for movie_id in eraser_similar_movies[:5]]

print("Top 5 similar movies to 'Heat':", heat_top_similar)
print("Top 5 similar movies to 'Eraser':", eraser_top_similar)

/Users/shambhavisingh/anaconda3/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


IndexError: index 0 is out of bounds for axis 0 with size 0